### Загрузка библиотек

In [187]:
import os
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import tqdm

from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

### Считывание файлов train

In [203]:
train = pd.read_csv("train.csv")
test = pd.read_csv("sample_submission.csv")

In [173]:
# train = train[:1000]

In [204]:
src_path = os.getcwd()
imgs = []
for img_name in tqdm(train.img_name.values, total=len(train)):
    img = cv2.imread(os.path.join("train", img_name))
    imgs.append(img)
imgs = np.array(imgs)

  0%|          | 0/13210 [00:00<?, ?it/s]

## Подготовка для обучения

### Признаки

In [175]:
# Среднее значение яркости изображения в цветовом пространстве HSV (оттенок, насыщенность, значение).
def brightness(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    v = hsv[..., 2].mean()
    return v / 255

# Резкость изображения на основе градиентов
def sharpness_grad_based(img):
    gradx = cv2.Sobel(img, cv2.CV_32F, 1, 0, ksize=1)
    grady = cv2.Sobel(img, cv2.CV_32F, 0, 1, ksize=1)
    grad = np.sqrt(gradx * gradx + grady * grady)
    return grad[1:-1, 1:-1].mean()


# Статические характеристики по цветовым каналам
def color_stats(img):
    mean_color = np.mean(img, axis=(0, 1))
    std_color = np.std(img, axis=(0, 1))
    return np.concatenate([mean_color, std_color])


# Статические характеристики по яркости
def brightness_stats(img):
    brightness_mean = np.mean(img)
    brightness_std = np.std(img)
    return np.array([brightness_mean, brightness_std])


def get_iqa_fv(img):
    fv = np.empty((0,))
    fv = np.append(fv, brightness(img))
    fv = np.append(fv, sharpness_grad_based(img))
    fv = np.append(fv, color_stats(img))
    fv = np.append(fv, brightness_stats(img))
    return fv


def get_features(list_of_imgs):
    features = []
    for img in tqdm(list_of_imgs):
        features.append(get_iqa_fv(img))
    features = np.vstack(features)
    return features

### Разделение на test и train

In [176]:
# Получение названий референсных картинок
ref_names = set()
for img_name in train.img_name.values:
    ref_names.add(img_name.split("_")[0])
ref_names = sorted(list(ref_names))
random.seed(7)
random.shuffle(ref_names)

In [177]:
# Деление на train и test
train_size = 0.8

train_names = []
train_ids = []
test_names = []
test_ids = []
for i, img_name in enumerate(train.img_name.values):
    ref = img_name.split("_")[0]
    if ref in ref_names[: int(train_size * len(ref_names))]:
        train_names.append(img_name)
        train_ids.append(i)
    else:
        test_names.append(img_name)
        test_ids.append(i)

In [178]:
print("Подсчет признаков обучающей выборки...")
X_train = get_features(imgs[train_ids])
print("Подсчет признаков тестовой выборки...")
X_test = get_features(imgs[test_ids])
X_train.shape, X_test.shape

Подсчет признаков обучающей выборки...


  0%|          | 0/800 [00:00<?, ?it/s]

Подсчет признаков тестовой выборки...


  0%|          | 0/200 [00:00<?, ?it/s]

((800, 10), (200, 10))

In [179]:
train_df = train.set_index("img_name").loc[train_names]
test_df = train.set_index("img_name").loc[test_names]

y_train = train_df.PSNR.values
y_test = test_df.PSNR.values

## Модель и обучение

In [194]:
param_grid = {
    "n_estimators": [80, 100, 150],
    "learning_rate": [0.2, 0.3, 0.4],
    "max_depth": [5, 6, 7],
    "min_child_weight": [1, 3, 5],
    "subsample": [0.9, 1.0],
    "colsample_bytree": [0.9, 1.0],
    "gamma": [0, 0.1],
}

model = XGBRegressor()

grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=5,
    scoring="neg_mean_absolute_error",
    verbose=1,
    return_train_score=True
)

grid_search.fit(X_train, y_train)

# Лучшие параметры
best_params = grid_search.best_params_

Fitting 5 folds for each of 648 candidates, totalling 3240 fits


In [196]:
# best_params = {
#     "colsample_bytree": 0.9,
#     "gamma": 0.1,
#     "learning_rate": 0.2,
#     "max_depth": 7,
#     "min_child_weight": 1,
#     "n_estimators": 100,
#     "subsample": 0.9,
# }

In [197]:
model = XGBRegressor(**best_params)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Mean Absolute Error:", mean_absolute_error(y_test, y_pred))
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
print("R2 score:", r2_score(y_test, y_pred))

## Финальный результат

In [184]:
src_path = os.getcwd()
test_imgs = []
for img_name in tqdm(test.img_name.values, total=len(test)):
    img = cv2.imread(os.path.join("test", img_name))
    test_imgs.append(img)
test_imgs = np.array(test_imgs)

In [185]:
print("Подсчет признаков тестовой выборки...")
test_features = get_features(test_imgs)

In [205]:
X_final = get_features(imgs)
y_final = train.PSNR.values

model = XGBRegressor(**best_params)
model.fit(X_final, y_final)

  0%|          | 0/13210 [00:00<?, ?it/s]

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.1, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.2, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=7, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [206]:
y_pred = model.predict(test_features)

In [207]:
# Сохранение результатов
ans = pd.DataFrame({"img_name": test.img_name.values, "PSNR": y_pred})
ans.to_csv("predict 5.csv", index=False)